In [1]:
import pandas as pd

In [4]:
payments_df = pd.read_csv('Data/payments.csv',index_col=[0])
payments_df

,transaction_id,contract_id,client_id,transaction_date,payment_amt,payment_code
stt,,,,,,
0,20175,927,1,2018-05-22 18:08:31,66.66,PAYMENT
1,8485,927,1,2017-11-26 17:08:15,66.66,PAYMENT
2,13778,927,1,2018-02-22 17:08:23,66.66,PAYMENT
3,22768,927,1,2018-06-24 18:08:44,66.66,PAYMENT
4,15698,927,1,2018-03-22 17:08:24,66.66,PAYMENT
...,...,...,...,...,...,...
25554,25075,1603,1280,2018-07-19 18:09:24,1666.68,PAYMENT
25555,24711,1603,1280,2018-07-16 18:09:20,0.01,PAYMENT
25556,25076,1603,1280,2018-07-19 18:09:24,64.99,PAYMENT


In [11]:
# Due to an internal error, let's try the calculation again with a corrected approach

# First, correct and reformat payment_amt and transaction_date
payments_df['payment_amt'] = payments_df['payment_amt'].str.replace(',', '.').astype(float)
payments_df['transaction_date'] = pd.to_datetime(payments_df['transaction_date'])

# Calculating the required fields
grouped = payments_df.groupby('contract_id').agg(
    transaction_count=('transaction_id', 'count'),
    client_id=('client_id', 'first'),
    contract_start=('transaction_date', 'min'),
    contract_end=('transaction_date', 'max'),
    total_payment_amt=('payment_amt', lambda x: x[payments_df.loc[x.index, 'payment_code'] == 'PAYMENT'].sum()),
    average_default_amt=('payment_amt', lambda x: x[payments_df.loc[x.index, 'payment_code'] == 'DEFAULT'].mean()),
    default_rate=('payment_code', lambda x: (x == 'DEFAULT').mean())
)

# Calculate contract duration in days
grouped['contract_duration_days'] = (grouped['contract_end'] - grouped['contract_start']).dt.days

# Calculate transaction frequency per day for each contract
grouped['transaction_frequency_per_day'] = grouped['contract_duration_days'] / grouped['transaction_count']

# Fill NaN values with 0 for average_default_amt and default_rate to handle contracts without defaults
grouped['average_default_amt'] = grouped['average_default_amt'].fillna(0)
grouped['default_rate'] = grouped['default_rate'].fillna(0)

grouped.reset_index(inplace=True)

grouped.head()

,contract_id,transaction_count,client_id,contract_start,contract_end,total_payment_amt,average_default_amt,default_rate,contract_duration_days,transaction_frequency_per_day
0,1,31,248,2017-07-03 18:08:05,2018-07-23 18:08:07,1390.00,60.0,0.129032,385,12.419355
1,2,22,653,2017-07-30 18:08:40,2018-05-20 18:08:30,1327.00,0.0,0.000000,293,13.318182
2,3,12,90,2017-07-17 18:08:25,2018-07-23 18:08:07,5367.00,10.0,0.166667,370,30.833333
3,4,1,134,2018-02-19 17:08:22,2018-02-19 17:08:22,0.01,0.0,0.000000,0,0.000000
4,5,25,545,2017-07-03 18:08:05,2018-07-15 18:09:18,7060.00,160.0,0.040000,377,15.080000


In [ ]:
grouped.to_csv('Data/contracts.csv',index=False)

In [13]:
payments_df.to_csv('Data/payments.csv',index=False)